In [288]:
import importlib
import hp
importlib.reload(hp)
from hp import look_df
from hp import check_dups

import mom 
importlib.reload(mom)

import pandas as pd
import os
import urllib.request
import tarfile
import numpy as np

In [289]:
if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("data/DM"):
    os.makedirs("data/DM")

if not os.listdir("data/DM"):
    print('Downloading and extracting data...')
    thetarfile = "http://kentdaniel.net/data/momentum/DM_data_2017_03.tar.gz"
    ftpstream = urllib.request.urlopen(thetarfile)
    thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
    thetarfile.extractall(path="data/DM")

In [290]:
########################################################################
# Load daily momentum data
########################################################################
col_names = ['date', 'decile', 'ret', 'me', 'num']
dfMomDly = pd.read_csv('data/DM/d_m_pt_tot.txt', delimiter=r'\s+', header=None, names=col_names)
dfMomDly = mom.calcLongShort(dfMomDly, bin_col='decile', return_col='ret')
dfMomDly = dfMomDly[dfMomDly['decile'] == 11].copy()
dfMomDly.reset_index(drop=True, inplace=True)
dfMomDly.drop(columns=['decile'], inplace=True)

########################################################################
# Calculate scaled volatility factor (columname  scldfctVol)
########################################################################
dfMomDly['ret2'] = dfMomDly['ret']**2
dfMomDly['rldsum'] = dfMomDly['ret2'].rolling(window=126).sum()
dfMomDly['rldsumMnth']= dfMomDly['rldsum'] / 126 * 21
dfMomDly['fcstdfVol'] = np.sqrt(dfMomDly['rldsum']) # forecasted vol
dfMomDly['scldfctVol'] = 0.12 / dfMomDly['fcstdfVol'] # scaled factor vol
dfVol = dfMomDly[['date', 'scldfctVol']].copy()
dfVol.dropna(inplace=True)
dfVol.reset_index(drop=True, inplace=True)
look_df(dfVol, 10)

Number of rows: 23,614
Number of columns: 2

First 10 rows:


,date,scldfctVol
0,19270602,1.032954
1,19270603,1.037766
2,19270604,1.041746
3,19270606,1.044220
4,19270607,1.034296
5,19270608,1.034244
6,19270609,1.033761
7,19270610,1.030805
8,19270611,1.028429
9,19270614,1.032924


In [291]:
########################################################################
# Calculate long short momentum portfolio monthly returns
########################################################################

dfMom = mom.calcMomDeciles(reload=False)
dfMom = mom.calcLongShort(dfMom, bin_col='DM_Decile', return_col='DM_Ret')
dfMomLS = dfMom[dfMom['DM_Decile'] == 11].copy()
dfMomLS.reset_index(drop=True, inplace=True)
dfMomLS.drop(columns=['DM_Decile'], inplace=True)
dfMomLS.rename(columns={'DM_Ret': 'momRet'}, inplace=True)
look_df(dfMomLS, 5)

Number of rows: 1,164
Number of columns: 2

First 5 rows:


,date,momRet
0,1927-01,0.030043
1,1927-02,-0.005318
2,1927-03,0.086889
3,1927-04,0.037577
4,1927-05,0.036419


In [292]:
########################################################################
# Select last date of each month to match monthly frequency
########################################################################
dfVol['date'] = pd.to_datetime(dfVol['date'], format='%Y%m%d')
dfVol['period'] = dfVol['date'].dt.to_period('M')
last_dates_dfVol = dfVol.groupby('period')['date'].max().reset_index()
dfVolEOM = pd.merge(last_dates_dfVol, dfVol, on=['date', 'period'], how='left')
dfVolEOM.drop(columns=['date'], inplace=True)
dfVolEOM.rename(columns={'period': 'date'}, inplace=True)
dfVolEOM['date'] = dfVolEOM['date'] + 1 # need to shift date to match with mom data
look_df(dfVolEOM, 5)

Number of rows: 1,075
Number of columns: 2

First 5 rows:


,date,scldfctVol
0,1927-07,1.033845
1,1927-08,1.030880
2,1927-09,1.031423
3,1927-10,1.079664
4,1927-11,1.010692


In [293]:
########################################################################
# Merge momentum monthly return data and volatility scaling factor
########################################################################
dfMomLS = pd.merge(dfMomLS, dfVolEOM, on='date', how='inner')
dfMomLS['momRetScld'] = dfMomLS['momRet'] * dfMomLS['scldfctVol']
look_df(dfMomLS, 20)

Number of rows: 1,075
Number of columns: 4

First 20 rows:


,date,momRet,scldfctVol,momRetScld
0,1927-07,0.091263,1.033845,0.094352
1,1927-08,0.060550,1.030880,0.062420
2,1927-09,0.076771,1.031423,0.079183
3,1927-10,0.008860,1.079664,0.009565
4,1927-11,-0.066919,1.010692,-0.067635
5,1927-12,0.003944,0.933467,0.003681
6,1928-01,-0.005409,0.937127,-0.005069
7,1928-02,0.007226,1.003193,0.007249
8,1928-03,0.055430,1.050363,0.058222
9,1928-04,-0.044439,1.091827,-0.048520


In [294]:
########################################################################
# Calculate excess returns for both momRet and momRetScld
########################################################################
dfMomExcess = mom.calcExRet(df = dfMomLS, return_col = 'momRet', isDatePeriod=True) 
dfMomExcess = mom.calcExRet(df = dfMomExcess, return_col = 'momRetScld', isDatePeriod=True)
look_df(dfMomExcess, 20)

/Users/kevinnecochea/Documents/edu/ucla/2024-spring/qam/HW/FP/mom.py:301: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = rf_reader.read()[0] / 100
/Users/kevinnecochea/Documents/edu/ucla/2024-spring/qam/HW/FP/mom.py:301: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = rf_reader.read()[0] / 100


Number of rows: 1,075
Number of columns: 4

First 20 rows:


/Users/kevinnecochea/Documents/edu/ucla/2024-spring/qam/HW/FP/mom.py:301: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = rf_reader.read()[0] / 100
/Users/kevinnecochea/Documents/edu/ucla/2024-spring/qam/HW/FP/mom.py:301: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = rf_reader.read()[0] / 100


,date,momRet,scldfctVol,momRetScld
0,1927-07,0.088263,1.033845,0.091352
1,1927-08,0.057750,1.030880,0.059620
2,1927-09,0.074671,1.031423,0.077083
3,1927-10,0.006360,1.079664,0.007065
4,1927-11,-0.069019,1.010692,-0.069735
5,1927-12,0.001744,0.933467,0.001481
6,1928-01,-0.007909,0.937127,-0.007569
7,1928-02,0.003926,1.003193,0.003949
8,1928-03,0.052530,1.050363,0.055322
9,1928-04,-0.046639,1.091827,-0.050720


In [295]:
########################################################################
# Calculate excess returns for both momRet and momRetScld
########################################################################

# Define the statistics functions
def excess_return(x):
    return x.mean() * 100 * 12

def volatility(x):
    return x.std() * 100 * np.sqrt(12)

def sharpe_ratio(x):
    std_dev = x.std()
    return (x.mean() * 100 * 12) / (std_dev * 100 * np.sqrt(12)) if std_dev != 0 else np.nan

def skewness(x):
    return x.skew()

# Apply the statistics functions to each stock return column
stats = {
    'Excess_Return': {
        'momRet': excess_return(dfMomExcess['momRet']),
        'momRetScld': excess_return(dfMomExcess['momRetScld'])
    },
    'Volatility': {
        'momRet': volatility(dfMomExcess['momRet']),
        'momRetScld': volatility(dfMomExcess['momRetScld'])
    },
    'Sharpe_Ratio': {
        'momRet': sharpe_ratio(dfMomExcess['momRet']),
        'momRetScld': sharpe_ratio(dfMomExcess['momRetScld'])
    },
    'Skewness': {
        'momRet': skewness(dfMomExcess['momRet']),
        'momRetScld': skewness(dfMomExcess['momRetScld'])
    }
}

# Convert the stats dictionary to a DataFrame for better visualization
stats_dfMomExcess = pd.DataFrame(stats)
look_df(stats_dfMomExcess, 10)


Number of rows: 2
Number of columns: 4

First 10 rows:


,Excess_Return,Volatility,Sharpe_Ratio,Skewness
momRet,14.122547,29.896932,0.472374,-1.996351
momRetScld,20.149060,23.377840,0.861887,-0.452561
